In [2]:
# 신용점수 외의 다른 값이 모두 전처리되고 join된 자료 불러오기
import pandas as pd
joined = pd.read_csv('joined.csv',index_col=0)

/Users/jinwon/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# tree에 넣기 위해 전처리
joined['birth_year'] = joined['birth_year'].apply(lambda x: int(str(x)[0:4]))
joined['yearly_income'] = joined['yearly_income'].apply(lambda t: int(t)/1000000)
joined['desired_amount'] = joined['desired_amount'].apply(lambda t: int(t)/1000000)
joined['existing_loan_amt'] = joined['existing_loan_amt'].apply(lambda t: int(t)/1000000)
joined['existing_loan_cnt'] = joined['existing_loan_cnt'].apply(lambda t: int(float(t)))

In [4]:
# 범주형 자료를 one-hot encoding하고 회귀나무에 넣을 수 있는 feature만 선택
treejoined = pd.concat(
    [
        joined,
        pd.get_dummies(joined.income_type, prefix='income'),
        pd.get_dummies(joined.houseown_type, prefix='house')
    ],
    axis=1
).reset_index(drop=True).drop(
    [
        "application_id", "loanapply_insert_time",
        'insert_time',"bank_id", "product_id",
        'income_type','loan_rate','loan_limit',
        'is_applied','user_id','company_enter_month',
        'purpose','employment_type','houseown_type'
    ],
    axis=1
)

In [6]:
# 신용점수가 기록된 자료와 null인 자료 분리
traincredit = treejoined[treejoined['credit_score'].notna()]
trainnullcredit = treejoined[treejoined['credit_score'].isna()]
treejoined = pd.DataFrame()

In [8]:
# sklearn으로 데이터를 학습/검증 데이터 분할하고, tree모델을 적합시킨다.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
X = traincredit.drop(['credit_score'], axis=1)
y = traincredit['credit_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)
model = DecisionTreeRegressor(random_state=44)
model.fit(X_train, y_train)

In [11]:
# 적합시킨 tree에 신용점수가 null인 자료를 넣어 회귀값을 구한다.
predictions = model.predict(trainnullcredit.drop(['credit_score'],axis = 1))
joined = pd.read_csv('joined.csv',index_col=0)
nullcredit = joined[joined['credit_score'].isna()]
nullcredit['credit_score'] = predictions

/Users/jinwon/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
<ipython-input-11-ad5d0805df6e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullcredit['credit_score'] = predictions


In [14]:
# 나머지 데이터에 맞게 일의 자리는 버림한다.
nullcredit['credit_score']= nullcredit['credit_score']//10*10

<ipython-input-14-d206d4141f35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nullcredit['credit_score']= nullcredit['credit_score']//10*10


In [13]:
# 기존 데이터와 합쳐 내보내기한다.
fulljoined = pd.concat([joined[joined['credit_score'].notna()],nullcredit])
fulljoined.to_csv("fulljoined.csv")